<a href="https://colab.research.google.com/github/Pramurta/Neural-Nets-Assignment-2/blob/main/Neural_Nets_Assignment_2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -cq http://opensurfaces.cs.cornell.edu/static/minc/minc-2500.tar.gz 

In [2]:
import tarfile
tar = tarfile.open("minc-2500.tar.gz")
tar.extractall()
tar.close()

In [3]:
original_images_dir = '/content/minc-2500/images'
sub_dirs = ['metal', 'glass', 'fabric', 'leather', 'paper', 'stone', 'wood', 'plastic', 'water', 'foliage']

In [5]:
import os
train_dir = '/content/train'
val_dir = '/content/val'
for sub_dir in sub_dirs:
    os.makedirs(train_dir+"/"+sub_dir, exist_ok=True)
for sub_dir in sub_dirs:
    os.makedirs(val_dir+"/"+sub_dir, exist_ok=True)

In [6]:
import cv2
for sub_dir in sub_dirs:
    cur_folder = original_images_dir+"/"+sub_dir
    for i,filename in enumerate(os.listdir(cur_folder)):
        img = cv2.imread(cur_folder+"/"+filename)
        if i>1999:
            cv2.imwrite(val_dir+"/"+sub_dir+"/"+filename, img)
        else:
            cv2.imwrite(train_dir+"/"+sub_dir+"/"+filename, img)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255 and image augmentation has also been used to reduce variance in our training
train_datagen = ImageDataGenerator(
      rescale=1./255,
      zoom_range=0.3,
      horizontal_flip=True,
      brightness_range=[0.4,1.5],
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 140 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/train/',  # This is the source directory for training images
        target_size=(150, 150),
        batch_size=64,
        class_mode='categorical')

# Flow validation images in batches of 30 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/val/',  # This is the source directory for validation images
        target_size=(150, 150),  
        batch_size=64,
        class_mode='categorical')

Found 20000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [8]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

  
from tensorflow.keras.applications.inception_v3 import InceptionV3

#Employing transfer learning by utilizing the pretrained weights of Inception V3 model up to layer "mixed9" trained on the imagenet dataset 
pre_trained_model_inception = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = "imagenet")


for layer in pre_trained_model_inception.layers:
    layer.trainable = False
  

last_layer = pre_trained_model_inception.get_layer('mixed9')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

87924736/87910968 [==============================] - 1s 0us/step
last layer output shape:  (None, 3, 3, 2048)


In [9]:
from tensorflow.keras.optimizers import Adam


# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 8192 hidden units and ReLU activation
x = layers.Dense(8192, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a fully connected layer with 2048 hidden units and ReLU activation   
x = layers.Dense(2048, activation='relu')(x) 
 # Add a dropout rate of 0.1
x = layers.Dropout(0.1)(x) 
x = layers.Dense(512, activation='relu')(x)           
# Add a final softmax layer for classification
x = layers.Dense(10, activation='softmax')(x)           

model = Model(pre_trained_model_inception.input, x) 

opt = Adam(learning_rate=0.001)

model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=100, steps_per_epoch=313, validation_data = validation_generator, verbose = 1, validation_steps=79)

Epoch 1/100
199/313 [==================>...........] - ETA: 7:55 - loss: 1.4634 - accuracy: 0.5961